In [175]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil

def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

    return len(failed_downloads) == 0  # 모든 다운로드가 성공했으면 True 반환



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    failed_posts = []

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            print(f"Processing: {title}__{user}")
            success = download_images_from_links(driver, href, download_path, title)
            is_successful_zip = compress_images_to_zip(download_path, href, user)
            if not is_successful_zip or not success:
                failed_posts.append((href,title,user))

    print("_____finished______")
    print("Failed posts:")
    for title, url, user in failed_posts:
        print(f"Title: {title}, URL: {url}")

    return failed_posts



    


In [158]:

download_path = '/Users/jonathanoh/Desktop/down'
excel_path = '/Users/jonathanoh/Desktop/down/image_url.xlsx'

options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')

In [159]:


# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
# post_list.reverse()

post_list

#process_images(driver, post_list, download_path)

[('https://arca.live/b/aiartreal/94721143?p=1', '랜덤모델 테스트', 'adoniel'),
 ('https://arca.live/b/aiartreal/94721043?p=1', '잘 자다', 'SugarSweet'),
 ('https://arca.live/b/aiartreal/94720510?p=1', '리스팩트님 와카 공유2차 굳...', '냥이집사'),
 ('https://arca.live/b/aiartreal/94720208?p=1',
  'webui 버전업 문제로 재설치 5회차',
  'carmen'),
 ('https://arca.live/b/aiartreal/94719230?p=1', '오픈포즈 2400개 정도..', '마린2세'),
 ('https://arca.live/b/aiartreal/94719228?p=1',
  '학습첸은 아니지만 그래도 알고 배운곳이 여기라.. 드림부스 학습시에',
  '니라라'),
 ('https://arca.live/b/aiartreal/94718928?p=1',
  '클립챔프 같은 무료 동영상제작프로그램...',
  '냥이집사'),
 ('https://arca.live/b/aiartreal/94717312?p=1',
  'cynthiaR0_1124 수영복 찍먹',
  'carmen'),
 ('https://arca.live/b/aiartreal/94717056?p=1',
  'cynthiaK9_1124 수영복 찍먹',
  'carmen'),
 ('https://arca.live/b/aiartreal/94716884?p=1', 'AI로 생성이 될까싶은 이미지', '젬마'),
 ('https://arca.live/b/aiartreal/94716640?p=1', '플레이그라운드 프롬 돚거', '젬마'),
 ('https://arca.live/b/aiartreal/94716235?p=1', 'LCM LORA 테스트', 'Geeks'),
 ('https://arca.live/b/aiart

In [168]:
post_list[17]

('https://arca.live/b/aiartreal/94712801?p=1', '서술형프롬프트 와카 공유 2차 192개', '리스펙트')

In [170]:
# list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)
# list = process_images(driver, list, download_path)
list = process_images(driver, [post_list[15]], download_path)


Processing: 가부키쵸__달의영혼
_____finished______
Failed posts:


In [151]:
print(len(post_list),"ddd",len(list))

45 ddd 1


In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [184]:

download_path = '/Users/jonathanoh/Desktop/down'
excel_path = '/Users/jonathanoh/Desktop/down/image_url.xlsx'

options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화

In [182]:
civitai_url = "https://civitai.com/images"
civitai_login = "https://civitai.com/login?returnUrl=/"

driver.get(civitai_login)

# login_button = driver.find_element(By.CLASS_NAME, 'mantine-Group-root mantine-1g4q40w')
# login_button.click()


# image_button = driver.find_element(By.CLASS_NAME, 'mantine-Text-root.mantine-xieqk9')

# # 요소 클릭
# image_button.click()

In [183]:
image_button = driver.find_element(By.CLASS_NAME, 'mantine-Text-root.mantine-xieqk9')

# 요소 클릭
image_button.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".mantine-Text-root.mantine-xieqk9"}
  (Session info: chrome=120.0.6099.109)
Stacktrace:
0   chromedriver                        0x0000000102fc44cc chromedriver + 4162764
1   chromedriver                        0x0000000102fbc654 chromedriver + 4130388
2   chromedriver                        0x0000000102c13bc0 chromedriver + 293824
3   chromedriver                        0x0000000102c5a040 chromedriver + 581696
4   chromedriver                        0x0000000102c99ab0 chromedriver + 842416
5   chromedriver                        0x0000000102c4d6bc chromedriver + 530108
6   chromedriver                        0x0000000102c4e930 chromedriver + 534832
7   chromedriver                        0x0000000102f89df8 chromedriver + 3923448
8   chromedriver                        0x0000000102f8e3cc chromedriver + 3941324
9   chromedriver                        0x0000000102f72028 chromed

In [286]:
import requests
from PIL import Image
import os

# Civitai API endpoint for images
api_url = "https://civitai.com/api/v1/images"
api_key = '3e027a878a60255d4913a0239e276306'

# Define parameters for popular images
# The actual parameters depend on the API documentation
params = {
    "sort": "Most Reactions",  # 예: 인기도에 따라 정렬
    "order": "desc",       # 내림차순 정렬 (가장 인기 있는 이미지부터)
    "limit": 200,    # 가져올 이미지 수
    "period": "Month"
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful
if response.ok:
    # Parse the JSON response
    images_data = response.json()
    print(images_data)

    # Extract image URLs or other relevant data
    image_urls = [image['url'] for image in images_data['items']] 
#     if 'data' in images_data else []
    
    # Print or process the image URLs
#     for url in image_urls:
#         print(url)
else:
    print("Failed to retrieve images. Status code:", response.status_code)
    
    
download_path = '/Users/jonathanoh/Desktop/down'

for item in images_data['items']:
    if 'url' in item:
        image_url = item['url']
        file_name = f"civitai_month_{item['id']}{os.path.splitext(image_url)[1]}"
        file_path = os.path.join(download_path, file_name)
        download_image(image_url, file_path)
        print(f"{file_name} 다운로드 완료.")
print('finish')


{'items': [{'id': 3895642, 'url': 'https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/b28a009c-adf8-444b-ab95-1c360dbb24b1/width=768/b28a009c-adf8-444b-ab95-1c360dbb24b1.jpeg', 'hash': 'UQLVzG^jE*%gpwIpR*gN.8s.%0x]~pxaoJ%M', 'width': 768, 'height': 1152, 'nsfwLevel': 'X', 'nsfw': True, 'createdAt': '2023-11-24T20:47:26.532Z', 'postId': 876230, 'stats': {'cryCount': 13, 'laughCount': 49, 'likeCount': 464, 'dislikeCount': 0, 'heartCount': 563, 'commentCount': 10}, 'meta': None, 'username': 'NerveoS'}, {'id': 3969838, 'url': 'https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/ae41e7e5-a3e3-4eed-936c-da31a8037455/width=1184/ae41e7e5-a3e3-4eed-936c-da31a8037455.jpeg', 'hash': 'U37,.2xu0M~B=^Io019aJB%1$%Iqxuxa^j-:', 'width': 1184, 'height': 1496, 'nsfwLevel': 'Mature', 'nsfw': True, 'createdAt': '2023-11-27T16:34:03.293Z', 'postId': 892289, 'stats': {'cryCount': 2, 'laughCount': 14, 'likeCount': 373, 'dislikeCount': 0, 'heartCount': 436, 'commentCount': 8}, 'meta': {'VAE': 'vae-ft-mse-840000-ema

In [234]:
def download_image(url, file_path):
    response = requests.get(url, stream=True)
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)


In [ ]:
import os
import requests

download_path = '/Users/jonathanoh/Desktop/down'

def download_image(url, file_path):
    response = requests.get(url)
    content_type = response.headers.get('content-type')
    if content_type == 'image/jpeg':
        file_path += '.jpeg'
    with open(file_path, 'wb') as file:
        file.write(response.content)

for item in images_data['items']:
    if 'url' in item:
        image_url = item['url']
        file_name = f"image_{item['id']}"
        file_path = os.path.join(download_path, file_name)
        download_image(image_url, file_path)
        print(f"{file_name} 다운로드 완료.")
print('finish')

In [235]:
download_path = '/Users/jonathanoh/Desktop/down'

for item in images_data['items']:
    if 'url' in item:
        image_url = item['url']
        file_name = f"civitai_month_{item['id']}{os.path.splitext(image_url)[1]}"
        file_path = os.path.join(download_path, file_name)
        download_image(image_url, file_path)
        print(f"{file_name} 다운로드 완료.")
print('finish')

civitai_month_3895642.jpeg 다운로드 완료.
civitai_month_3969838.jpeg 다운로드 완료.
civitai_month_3972024.jpeg 다운로드 완료.
civitai_month_3997714.jpeg 다운로드 완료.
civitai_month_4264356.jpeg 다운로드 완료.
civitai_month_3878307.jpeg 다운로드 완료.
civitai_month_3893278.jpeg 다운로드 완료.
civitai_month_4639936.jpeg 다운로드 완료.
civitai_month_4595384.jpeg 다운로드 완료.
civitai_month_3966422.jpeg 다운로드 완료.
civitai_month_3991654.jpeg 다운로드 완료.
civitai_month_4586505.jpeg 다운로드 완료.
civitai_month_3924105.jpeg 다운로드 완료.
civitai_month_4264369.jpeg 다운로드 완료.
civitai_month_4022727.jpeg 다운로드 완료.
civitai_month_3982383.jpeg 다운로드 완료.
civitai_month_4311166.jpeg 다운로드 완료.
civitai_month_4588004.jpeg 다운로드 완료.
civitai_month_4645442.jpeg 다운로드 완료.
civitai_month_3999713.jpeg 다운로드 완료.
civitai_month_4403456.jpeg 다운로드 완료.
civitai_month_3963234.jpeg 다운로드 완료.
civitai_month_3844531.jpeg 다운로드 완료.
civitai_month_4585831.jpeg 다운로드 완료.
civitai_month_4612198.jpeg 다운로드 완료.
civitai_month_4338941.jpeg 다운로드 완료.
civitai_month_3924526.jpeg 다운로드 완료.
civitai_month_4125217.jpeg 다

In [343]:
import os
import exifread
import pandas as pd



def get_exif_data(image_path):
    """
    이미지 파일로부터 Exif 데이터를 추출합니다.
    """
    try:
        with open(image_path, 'rb') as f:
            exif_data = exifread.process_file(f)
        return {tag: str(exif_data[tag]) for tag in exif_data.keys() if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote')}
    except Exception as e:
        print(f"Error reading EXIF data for {image_path}: {e}")
        return {}

def process_folder(folder_path):
    """
    주어진 폴더 내의 모든 이미지 파일에 대한 Exif 데이터를 수집합니다.
    """
    exif_data_list = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff')):
            file_path = os.path.join(folder_path, filename)
            exif_data = get_exif_data(file_path)
            exif_data['Filename'] = filename  # 파일명 추가
            exif_data_list.append(exif_data)

    return exif_data_list

def save_to_excel(data, excel_path):
    """
    수집된 데이터를 엑셀 파일로 저장합니다.
    """
    df = pd.DataFrame(data)
    df.to_excel(excel_path, index=False)

# 폴더 경로
folder_path = '/Users/jonathanoh/Desktop/down/civitai_231223' 
# 이미지 파일의 EXIF 데이터 수집
exif_data_list = process_folder(folder_path)

# 결과를 엑셀 파일로 저장
excel_path = '/Users/jonathanoh/Desktop/down/exif_data.xlsx'
save_to_excel(exif_data_list, excel_path)


PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.


In [325]:
import exifread

def get_exif_data(image_path):
    """
    이미지 파일로부터 Exif 데이터를 추출하여 반환합니다.
    """
    with open(image_path, 'rb') as f:
        exif_data = exifread.process_file(f)
    return exif_data

def print_exif_data(exif_data):
    """
    Exif 데이터를 출력합니다.
    """
    for tag in exif_data.keys():
        if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
            print(f"{tag}: {exif_data[tag]}")

            
for file_name in os.listdir(image_directory):
    print_exif_data(get_exif_data(os.path.join(image_directory, file_name)))

PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
File format not recognized.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.


Image ExifOffset: 26
EXIF UserComment: (best quality:1.1), shrex, forest, full-body,  <lora:shrek:1> <lora:leg_slider_v1_10:3>Negative prompt: (worst quality, low quality,:1.15),Steps: 25, Sampler: dpmpp_2m_sde_gpu_karras, CFG Scale: 7.0, Seed: 0, Size: 512x512, Model hash: 5dd07a46d7, Model: ANIME\richyrichmix_V2Fp16, Version: ComfyUI
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: 
Image ExifOffset: 26
EXIF UserComment: Dreamlike sky, floating balloons made of newspaper clippings, surreal landscape below, retro-futuristic aesthetic, (hyperrealistic facial features:1.3) Negative prompt: plain background, platform, ugly, deformed, noisy, low poly, blurry, disfigured, studio, ai generatedSteps: 5, Sampler: dpmpp_sde_gpu, CFG scale: 2.0, Seed: 715939554932000, 

In [327]:
image_directory

'/Users/jonathanoh/Desktop/down/civitai_231223'

In [341]:
for file_name in os.listdir(image_directory):
#     image = Image.open(os.path.join(image_directory, file_name))
    image_path = os.path.join(image_directory, file_name)
    with open(image_path, 'rb') as f:
    # Exif 데이터 읽기
        tags = exifread.process_file(f)

# Exif 데이터 출력
    for tag in tags.keys():
        if tag =="EXIF UserComment":
            print(f"{tag}: {tags[tag]}")
            


PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
File format not recognized.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.
PNG file does not have exif data.


EXIF UserComment: (best quality:1.1), shrex, forest, full-body,  <lora:shrek:1> <lora:leg_slider_v1_10:3>Negative prompt: (worst quality, low quality,:1.15),Steps: 25, Sampler: dpmpp_2m_sde_gpu_karras, CFG Scale: 7.0, Seed: 0, Size: 512x512, Model hash: 5dd07a46d7, Model: ANIME\richyrichmix_V2Fp16, Version: ComfyUI
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: Dreamlike sky, floating balloons made of newspaper clippings, surreal landscape below, retro-futuristic aesthetic, (hyperrealistic facial features:1.3) Negative prompt: plain background, platform, ugly, deformed, noisy, low poly, blurry, disfigured, studio, ai generatedSteps: 5, Sampler: dpmpp_sde_gpu, CFG scale: 2.0, Seed: 715939554932000, Size: 1152x896, Model hash: 547EFB38, Model: pixelwaveturboExcellent_reborn.safetensors
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF UserComment: 
EXIF U